[this doc on github](https://github.com/dotnet/interactive/tree/master/samples/notebooks/fsharp/Docs)

# Formatting of Outputs in F# Notebooks

## Default formatting uses HTML

When you return a value or a display a value in a .NET notebook, the default formatting behavior is to try to provide some useful information about the object. If it's an array or other type implementing `IEnumerable`, that might look like this:

In [ ]:
display ["hello"; "world"]

[ 1 .. 4 ]

As you can see, by default a table structure is used whether you pass the object to the `display` method or return it as the cell's value.

### Switching to plain text

If you prefer, you can switch to plain text by default.  The simplest way to do this is to register `"text/plain"` as the default mime type for all objects and switch the `%A` printing:

In [ ]:
Formatter.SetPreferredMimeTypeFor(typeof<obj>, "text/plain")
Formatter.Register(fun (x:obj) (writer: TextWriter) -> fprintfn writer "%120A" x )

In [ ]:
[ 1 .. 4 ]

Now reset back to defaults:

In [ ]:
Formatter.ResetToDefault()

### Displaying objects

The default formatting behavior for many objects is to produce a table showing their properties and the values of those properties.

In [ ]:
type Person = { FirstName: string; LastName: string; Age: int }

// Evaluate a new person
{ FirstName = "Mitch"; LastName = "Buchannon"; Age = 42 }

### Displaying sequences

When you have a collection of objects, you can see the values listed for each item in the collection:

In [ ]:
let people =
    [
        { FirstName = "Mitch"; LastName = "Buchannon"; Age = 42 }
        { FirstName = "Hobie "; LastName = "Buchannon"; Age = 23 }
        { FirstName = "Summer"; LastName = "Quinn"; Age = 25 }
        { FirstName = "C.J."; LastName = "Parker"; Age = 23 }
    ]

people

### Displaying dictionaries

Similarly to the behavior for `IEnumerable` objects, you'll also see table output for dictionaries, but for each value in the dictionary, the key is provided rather than the index within the collection.

In [ ]:
// Cannot simply use 'dict' here, see https://github.com/dotnet/interactive/issues/12

let d = dict [("zero", 0); ("one", 1); ("two", 2)]
d

### HTML displays and deeply nested objects

Now let's try something a bit more complex. Let's look at a graph of objects. 

We'll redefine the `Person` class to allow a reference to a collection of other `Person` instances.

In [ ]:
type Person =
    { FirstName: string
      LastName: string
      Age: int
      Friends: ResizeArray<Person> }

let mitch = { FirstName = "Mitch"; LastName = "Buchannon"; Age = 42; Friends = ResizeArray() }
let hobie = { FirstName = "Hobie "; LastName = "Buchannon"; Age = 23; Friends = ResizeArray() }
let summer =  { FirstName = "Summer"; LastName = "Quinn"; Age = 25; Friends = ResizeArray() }

mitch.Friends.AddRange([ hobie; summer ])
hobie.Friends.AddRange([ mitch; summer ])
summer.Friends.AddRange([ mitch; hobie ])

let people = [ mitch; hobie; summer ]
display people

That's a bit hard to read, right?  The defaut formatting behaviors are not always as useful as they might be. In order to give you more control in these kinds of cases, formatters can be customized from within the .NET notebook.

### Registering custom plain text formatters

Let's clean up the output above by customizing the formatter for the `Person.Friends` property, which is creating a lot of noise. 

The way to do this is to use the `Formatter` API. This API lets you customize the formatting for a specific type. Since `Person.Friends` is a sequence of `Person` objects, i.e. type `seq<Person>`, we can register a custom formatter for that type to change the output. Let's just list their first names:

In [ ]:
Formatter<seq<Person>>.Register(
    mimeType = "text/plain",
    formatter = fun people writer ->
        for person in people do
            writer.Write(person.FirstName)
            writer.Write(" "))


Now display the `people` data again:

In [ ]:
people

You might have noticed that `people` is of type `ResizeArray<Person>`, but the table output still includes columns for `LastName`, `Age`, and `Friends`. What's going on here?

Notice that the custom formatter we just registered was registered for the mime type `"text/plain"`. The top-level formatter that's used when we call `display` requests output of mime type `"text/html"` and the nested objects are formatted using `"text/plain"`. It's the nested objects, not the top-level HTML table, that's using the custom formatter here.


### Registering custom HTML formatters

The HTML formatters apply by default to many objects. For example, to replace the default HTML table view for a particular collection type, you can register a formatter for the `"text/html"` mime type. Let's do that.   HTML is specified using an HTML DSL similar to Giraffe's ViewEngine.  See [Displaying output](Displaying-output.ipynb) for more details on the HTML DSL.

In this case we put the formatter registration in a module to prevent opening the `Html` DSL everywhere.

In [ ]:
module PersonHtmlFormatter = 
    
    // Locally open the F# HTML DSL.
    open Html

    Formatter<seq<Person>>.Register(
        mimeType = "text/html",
        formatter = Action<_,_>(fun (people: seq<Person>) (writer: TextWriter) ->
            table [] [
              thead [ _style ["color: blue"]] [
                th [] [ object "First Name" ]
                th [] [ object "Last Name" ]
                th [] [ object "Age" ]
              ]
              tbody [_style ["color: darkolivegreen"]] [
                for p in people do
                  tr [] [
                    td [] [ object p.FirstName ]
                    td [] [ object p.LastName ]
                    td [] [ object p.Age ; span [] [ str " years"  ] ] 
                  ]
              ]
            ]
            |> writer.Write))


Now display the `people` data again:

In [ ]:
people